In [1]:
# %%
#-----------------------------------------------------------------------------
model_name = "Qwen/Qwen1.5-0.5B-Chat-AWQ"
device = "cuda:0"
output_path = "speed.csv"
methods = ["baseline", "KiVi", "LogQuant", "PartialLogQuant"]
methods = ["KiVi"]
n_bit_set = [2]
full_precision_lengths = [128]

import sys
sys.path.append('../')
from src.LogQuant import (QuantoLogQuantizedCache, LogQuantizedCacheConfig, 
                          QuantoStreamingQuantizedCache, StreamingQuantizedCacheConfig, 
                          QuantoPartialStreamingQuantizedCache, PartialStreamingQuantizedCacheConfig,
                          QuantoPartialLogQuantizedCache, PartialLogQuantizedCacheConfig,
                          QuantoKiViSinkQuantizedCache, KiViSinkQuantizedCacheConfig)
#-----------------------------------------------------------------------------

import torch
from torch import nn
data_type = 'auto'
new_token_length = 4096

from datasets import load_dataset
ds = load_dataset("THUDM/LongBench", "2wikimqa_e", split="test")

# %%
from transformers import Cache, CacheConfig, QuantizedCacheConfig, DynamicCache, QuantizedCache, QuantoQuantizedCache
import torch

# %%
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import Cache, CacheConfig, QuantizedCacheConfig, QuantizedCache
import torch
from torch import nn

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=data_type,
    device_map=device,
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)


We suggest you to set `torch_dtype=torch.float16` for better efficiency with AWQ.
/opt/anaconda3/envs/torch2/lib/python3.11/site-packages/accelerate/utils/imports.py:306: UserWarning: Intel Extension for PyTorch 2.3 needs to work with PyTorch 2.3.*, but PyTorch 2.4.0 is found. Please switch to the matching version and run again.
  warnings.warn(


In [2]:
def to_chat_template(input_text, context):
    messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant."
    },
    {
        "role": "user",
        "content": f"fact: {context}\nquestion: {input_text}\n"
    }
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)



In [3]:
def get_cache(method, n_bits, dense):
    if method == "KiVi":
        config =  QuantizedCacheConfig(
            backend="quanto",
            nbits=n_bits,
            residual_length=dense,
            compute_dtype=data_type,
            device=device,
        )
        cache = QuantoQuantizedCache(config)
    elif method == "StreamingQuant":
        config =  StreamingQuantizedCacheConfig(
            backend="quanto",
            nbits=n_bits,
            window_length=(dense - 4)//2,
            sink_length=4,
            compute_dtype=data_type,
            device=device,
        )
        cache = QuantoStreamingQuantizedCache(config)
    elif method == "LogQuant":
        config =  LogQuantizedCacheConfig(
            backend="quanto",
            nbits=n_bits,
            window_length=dense//3,
            compute_dtype=data_type,
            device=device,
        )
        cache = QuantoLogQuantizedCache(config)
    elif method == "PartialStreamingQuant":
        config =  PartialStreamingQuantizedCacheConfig(
            backend="quanto",
            nbits=n_bits,
            window_length=int((dense - 4)/1.5),
            sink_length=4,
            compute_dtype=data_type,
            device=device,
        )
        cache = QuantoPartialStreamingQuantizedCache(config)
    elif method == "PartialLogQuant":
        config =  PartialLogQuantizedCacheConfig(
            backend="quanto",
            nbits=n_bits,
            window_length=dense//2,
            compute_dtype=data_type,
            device=device,
        )
        cache = QuantoPartialLogQuantizedCache(config)
    elif method == "KiViSink":
        config =  KiViSinkQuantizedCacheConfig(
            backend="quanto",
            nbits=n_bits,
            window_length=dense,
            compute_dtype=data_type,
            device=device,
        )
        cache = QuantoKiViSinkQuantizedCache(config)
    else:
        cache = None
    return cache

In [4]:
def run_with_cache(model, qid, cache, batch_size):
    chat_template = to_chat_template(ds[qid]["input"], ds[qid]["context"])
    input_ids = tokenizer([chat_template for _ in range(batch_size)], return_tensors="pt", padding=True, truncation=True).input_ids.to(device)
    length = input_ids.shape[1]
    output_lens = 0
    import time
    start = time.time()
    model.generation_config.temperature=None
    model.generation_config.top_p=None
    output = model.generate(input_ids, past_key_values=cache, pad_token_id=tokenizer.eos_token_id, max_new_tokens=new_token_length, do_sample=False)
    end = time.time()
    for out in output:
        output_lens += len(out)

    del input_ids
    del output
    
    return length, output_lens, end - start


In [16]:
import pandas as pd
import tqdm
df = pd.DataFrame(columns=["method", "n_bit", "dense", "batch_size", "qid", "input_len", "output_len", "time", "speed"])
for qid in tqdm.tqdm(range(10)):
    for method in methods:
        if method == "baseline":
            batch_sizes = [1, 2, 4, 8, 12, 16]
        else:
            batch_sizes = [1, 2, 4, 8, 12, 16, 24, 32, 48, 64, 80, 96, 112, 128]
        for n_bit in n_bit_set:
            for dense in full_precision_lengths:
                for batch_size in batch_sizes:
                    cache = get_cache(method, n_bit, dense)
                    length, output_lens, time = run_with_cache(model, qid, cache, batch_size)
                    df = pd.concat([df, pd.DataFrame([[method, n_bit, dense, batch_size, qid, length, output_lens, time, output_lens/time]], columns=df.columns)], ignore_index=True)
                    df.to_csv(output_path, index=False)

  0%|          | 0/10 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/opt/anaconda3/envs/torch2/lib/python3.11/site-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
